# Clustering the UCSD Campus

The University of California At San Diego has a very unique campus environment for its undergraduate students. The entire campus and undergraduate student population are currently divided into six residential colleges. When a student is admitted into UCSD, they are assigned to one of the 6 colleges, and that determines where they live, what general education classes they have to take, and what resources are available to them. These colleges are vital to UCSD's unique campus culture, which is what makes the UCSD campus, the perfect clustering problem. For this problem, I have decided to implement the unsupervised k-means algorithm to separate the campus into 9 clusters intended to be: Sixth College, Marshall College, Warren College, Muir College, Elanor Roosevelt College, Revelle College, the center of campus, the school of medicine, and the medical campus.

In [1]:
import pandas as pd
import numpy as np
import googlemaps
import time
from utils import distance, get_key
from visualize import cluster_map

## Data Wrangling

In order to cluster the campus, I need to first gather a dataset of names of places on campus and their locations. To do this, I have decided to use the Google Maps Places API and the Python client library for it. I first create a list of location categories that would be relevant to the UCSD campus. Then I search for each of those categories of locations one by one in the proximity of the campus and save the results to a list.

In [2]:
gmaps = googlemaps.Client(key=get_key())
gmaps

In [3]:
types = ["university", "art_gallery", "atm", "bakery", "bank", "bar", "beauty_salon", "bicycle_store", "book_store",
         "bus_station", "cafe", "campground", "clothing_store", "convenience_store", "doctor", "establishment",
         "food", "gym", "grocery_or_supermarket", "health", "hospital", "laundry", "library", "liquor_store",
         "local_government_office", "lodging", "meal_takeaway", "museum", "park", "parking", "pharmacy", "physiotherapist",
         "police", "post_office", "restaurant", "school", "stadium", "storage", "store", "supermarket", "transit_station"]

def page_search(loc, rad, category=''):
    """
    Returns metadata for places within radius of location meeting criteria for given category
    :param loc: tuple of longitude and latitude
    :param rad: integer for radius(in meters) to search
    :param category: string specifying category
    :return: dictionary of places' metadata
    """
    results = []
    search = gmaps.places_nearby(location=loc, rank_by="distance", type=category)
    results += search["results"]
    while "next_page_token" in search:
        time.sleep(2)
        search = gmaps.places_nearby(location=loc, page_token=search["next_page_token"], radius=rad, type=category)
        results += search["results"]       
    return results


places = []
for t in types:
    places += page_search((32.881439,-117.237729), 50, category=t)
places[0]

{'geometry': {'location': {'lat': 32.8811414, 'lng': -117.2376126},
  'viewport': {'northeast': {'lat': 32.88205658029149,
    'lng': -117.2362445197085},
   'southwest': {'lat': 32.87935861970849, 'lng': -117.2389424802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
 'id': 'b86d32f8f935290d78fba69272bca631527f9ca7',
 'name': 'Teaching + Learning Commons',
 'opening_hours': {'open_now': False},
 'place_id': 'ChIJGZma3MMG3IAR2kje8Xt0Fpo',
 'plus_code': {'compound_code': 'VQJ6+FX San Diego, California, United States',
  'global_code': '8544VQJ6+FX'},
 'reference': 'ChIJGZma3MMG3IAR2kje8Xt0Fpo',
 'scope': 'GOOGLE',
 'types': ['university', 'point_of_interest', 'establishment'],
 'vicinity': 'Geisel Library, Library Walk, San Diego'}

After gathering the data, I format the search results to be saved into a dataframe storing the names of locations, latitude and longitude.

In [4]:
dct = {"name": [], "latitude": [], "longitude": []}
for place in places:
    dct["name"].append(place["name"])
    dct["latitude"].append(place["geometry"]["location"]["lat"])
    dct["longitude"].append(place["geometry"]["location"]["lng"])
    
{key:dct[key][:3] for key in dct.keys()}

{'name': ['Teaching + Learning Commons',
  'Eucalyptus Point',
  'Thurgood Marshall College Lower Apartments'],
 'latitude': [32.8811414, 32.8814919, 32.88261579999999],
 'longitude': [-117.2376126, -117.2394106, -117.239135]}

In [5]:
df = pd.DataFrame(dct)
df.head()

,name,latitude,longitude
0,Teaching + Learning Commons,32.881141,-117.237613
1,Eucalyptus Point,32.881492,-117.239411
2,Thurgood Marshall College Lower Apartments,32.882616,-117.239135
3,Center for Research in Language,32.880529,-117.239434
4,UC San Diego Jacobs School of Engineering,32.881468,-117.235483


Now that I have a dataframe of places I can now filter the results of the api requests using truth values on the Pandas series to narrow our dataset to only locations on campus.

In [6]:
top_bound = ((df['latitude'] <= 32.8912) & (df['longitude'] <= -117.237248)) | \
            ((df['latitude'] <= 32.885216) & (df['longitude'] <= -117.222171)) | \
            ((df['latitude'] <= 32.882486) & (df['longitude'] <= -117.21923))
bottom_bound = (df['latitude'] >= 32.871570)
left_bound = df['longitude'] >= -117.243233
right_bound = df['longitude'] <= -117.218857

df = df[left_bound][right_bound][top_bound][bottom_bound]
df.head()

C:\Users\tanve\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,name,latitude,longitude
0,Teaching + Learning Commons,32.881141,-117.237613
1,Eucalyptus Point,32.881492,-117.239411
2,Thurgood Marshall College Lower Apartments,32.882616,-117.239135
3,Center for Research in Language,32.880529,-117.239434
4,UC San Diego Jacobs School of Engineering,32.881468,-117.235483


In [7]:
df.to_csv("places.csv")

In [8]:
places = {row[1]: (row[2], row[3]) for row in df.itertuples()}

{key:places[key] for key in list(places.keys())[:5]}

{'Teaching + Learning Commons': (32.8811414, -117.2376126),
 'Eucalyptus Point': (32.8815091, -117.2397034),
 'Thurgood Marshall College Lower Apartments': (32.88261579999999,
  -117.239135),
 'Center for Research in Language': (32.8805291, -117.2394335),
 'UC San Diego Jacobs School of Engineering': (32.8814678, -117.2354827)}

## K-Means

Now that our dataset has been put together, we can begin to implement the k-means algorithm. The goal of the algorithm is to take a sequence of points and seperate them into k clusters. The algorithm starts by randomly selecting a k number of points known as centroids. These centroids will act as the center of our final clusters. The algorithm then is comprised of 2 notable steps:

1. Update the clusters. We assign each point to a cluster based on which centroid the point is closest to. This step is performed by the **group_by_centroid** function which takes a sequence of places and sequence of centroids to return k clusters.
 

In [9]:
def group_by_centroid(places, centroids):
    """
    Assigns places to their respective closest centroids and returns a cluster of places for each centroid
    :param places: a sequence of places
    :param centroids: a sequence of centroids
    :return: a nested sequence containing sequences of places all closest to the same centroid
    """
    clusters = [[] for i in range(len(centroids))]
    for place_name, location in places.items():
        dists = [distance(centroid, location) for centroid in centroids]
        clusters[dists.index(min(dists))].append(location)
    return clusters

2. Update the centroids. We calculate the mean latitude and mean longitude of each cluster and those statistics are used as our new centroid. This step is performed by the **find_centroid** function which is mapped to compute on each cluster.



In [10]:
def find_centroid(cluster):
    """
    Returns updated centroid of given cluster
    :param cluster: a sequence of places
    :return: tuple of latitude and longitude for updated centroid
    """
    return tuple((np.mean([i[0] for i in cluster]), np.mean([i[1] for i in cluster])))

These 2 steps are then repeated until the centroids are unable to change or we reach a maxiumum update threshold we define ourselves.

In [11]:
def k_means(places, k, max_updates=100):
    """
    Uses the k-means algorithm to group places into k clusters
    :param places: a sequence of places
    :param k: amount of clusters to group places into
    :param max_updates: maximum number of centroid updates allowed
    :return: k number of centroids represented as a tuple of longitude and latitude
    """
    assert len(places) >= k, 'Not enough restaurants to cluster'
    
    old_centroids, n = [], 0
    indexes = list(np.random.choice(range(len(places)), size=k, replace=False))
    centroids = [list(places.values())[i] for i in indexes]

    while old_centroids != centroids and n < max_updates:
        old_centroids = centroids
        clusters = group_by_centroid(places, centroids)
        centroids = list(map(find_centroid, clusters))
        n += 1
    return centroids

In [12]:
centroids = k_means(places, 9)
centroids

[(32.88052192526316, -117.2357468894737),
 (32.87483578235294, -117.23580742941176),
 (32.87549527333333, -117.24083476333335),
 (32.87737832435898, -117.23459981282056),
 (32.877970555000005, -117.2242024875),
 (32.88328838064516, -117.22628617096777),
 (32.88605689333333, -117.24109096333335),
 (32.874998309677416, -117.23225843010754),
 (32.88062594255318, -117.24072253191487)]

# Visualization
Now it is time visualize how well the algorithm clustered our dataset. To visualize the algorithm, I run the following function which writes a json file containing metadata about our places and the centroids and starts a javascript application where our points are plotted on a map. Note that each distinct color of a point represents a different cluster.

**Disclaimer:** The visualization is in no way my original code. It is adapted from a recent DSC 20 class project.

In [ ]:
cluster_map(places, centroids)

Serving HTTP on 0.0.0.0 port 8000 ...
Type Ctrl-C to exit.


<img src="map.png">

# Conclusion
The K-means algorithm, unfortunately, is not a very sophisticated clustering algorithm compared to industry standard solutions like dbscan and gaussian mixture models. Because of how the algorithm starts by picking random locations, that randomness makes each runtime of the algorithm return different clustering results. Above is a screenshot of one particular runtime that performed very well, while at the same time highlighting reasoning for the algorithm's results. The algorithm was able to autonomously separate the campus by all the subdivisions mentioned in the introduction except for Sixth College and The School of Medicine. This was largely a result of Google Maps lacking a lot of data on the residential areas of Sixth College so it was difficult for the algorithm to identify it as a cluster. Because of that Sixth College and half of The School of Medicine were classified as a single cluster, and the other half of the School of Medicine was its own cluster.

Despite the inconsistent results of the k-means clustering approach, the UCSD campus was still a great clustering dataset problem. I will revisit this problem in the future, but when I do I plan on doing 2 things to improve success: collect a more detailed dataset, and implement a more sophisticated clustering algorithm